## Exploring the SQUADv2 dataset using Autolabel

#### Setup the API Keys for providers that you want to use

In [1]:
import os

# provide your own OpenAI API key here
os.environ['OPENAI_API_KEY'] = 'sk-xxxxxxxxxxxxxxxxx'

#### Install the autolabel library

In [2]:
!pip3 install 'refuel-autolabel[openai]'

#### Download the dataset

In [3]:
from autolabel import get_data

get_data('squad_v2')

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100% [........................................................] 177008 / 177008

100% [......................................................] 1786563 / 1786563

This downloads two datasets:
* `test.csv`: This is the larger dataset we are trying to label using LLMs
* `seed.csv`: This is a small dataset where we already have human-provided labels

## Start the labeling process!

Labeling with Autolabel is a 3-step process:
* First, we specify a labeling configuration (see `config.json` below)
* Next, we do a dry-run on our dataset using the LLM specified in `config.json` by running `agent.plan`
* Finally, we run the labeling with `agent.run`

In [5]:
import json

from autolabel import LabelingAgent

In [33]:
# load the config
with open('config_squad_v2.json', 'r') as f:
     config = json.load(f)

Let's review the configuration file below. You'll notice the following useful keys:
* `task_type`: `question_answering` (since it's a question answering task)
* `model`: `{'provider': 'openai', 'name': 'gpt-3.5-turbo'}` (use a specific OpenAI model)
* `prompt.task_guidelines`: `'You are an expert at answering questions based on wikipedia articles` (how we describe the task to the LLM)
* `prompt.few_shot_num`: 3 (how many labeled examples to provide to the LLM)

In [34]:
config

{'task_name': 'OpenbookQAWikipedia',
 'task_type': 'question_answering',
 'dataset': {'label_column': 'answer', 'delimiter': ','},
 'model': {'provider': 'openai', 'name': 'gpt-3.5-turbo'},
 'prompt': {'task_guidelines': 'You are an expert at answering questions based on wikipedia articles. Your job is to answer the following questions using the context provided with the question. Use the context to answer the question - the answer is a continuous span of words from the context.\n',
  'output_guidelines': 'Your answer will consist of an explanation, followed by the correct answer. The last line of the response should always be is JSON format with one key: {"label": "the correct answer"}.\n If the question cannot be answered using the context and the context alone without any outside knowledge, the question is unanswerable. If the question is unanswerable, return the answer as {"label": "unanswerable"}\n',
  'few_shot_examples': 'seed.csv',
  'few_shot_selection': 'semantic_similarity',

In [22]:
# generate explanations for chain of thought
seed_with_explanations = agent.generate_explanations('seed.csv')

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

2023-06-14 15:59:48 openai INFO: error_code=None error_message='That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 87d1cac9c00b5c4994a0e5fe446a2087 in your message.)' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
2023-06-14 15:59:48 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 87d1cac9c00b5c4994a0e5fe446a2087 in your message.).


In [23]:
# # Use seedset with explanations in config
# config['prompt']['few_shot_examples'] = seed_with_explanations

In [35]:
# create an agent for labeling
agent = LabelingAgent(config=config)

In [25]:
agent.plan('test.csv')

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $8.7028 │
│ Number of Examples       │ 2000    │
│ Average cost per example │ $0.0044 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at answering questions based on wikipedia articles. Your job is to answer the following questions using the context provided with the question. Use the context to answer the question - the answer is a continuous span of words from the context.


Your answer will consist of an explanation, followed by the correct answer. The last line of the response should always be is JSON format with one key: {"label": "the correct answer"}.
 If the question cannot be answered using the context and the context alone without any outside knowledge, the question is unanswerable. If the question is unanswerable, return the answer as {"label": "unanswerable"}


Some examples with their output answers are provided below:

Context: The final major evolution of the steam engine design was the use of steam turbines starting in the late part of the 19th century. Steam turbines are generally more efficient than reciprocating piston type steam engines (for outputs above several hundred horsepow

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [36]:
labels, df, metrics_list = agent.run('test.csv', max_items=100)

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

2023-06-14 16:28:05 openai INFO: error_code=None error_message='The server had an error while processing your request. Sorry about that!' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
2023-06-14 16:28:05 langchain.embeddings.openai WARNING: Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.


/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Actual Cost: 0.1827


┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ f1     ┃ support ┃ threshold ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 0.5474 │ 100     │ -inf      │ 0.21     │ 1.0             │
└────────┴─────────┴───────────┴──────────┴─────────────────┘

Total number of failures: 0

We are at 21% accuracy when labeling the first 100 examples. Let's see if we can use confidence scores to improve accuracy further by removing the less confident examples from our labeled set.

### Compute confidence scores


In [47]:
# Start computing confidence scores (using Refuel's LLMs)
os.environ['REFUEL_API_KEY'] = 'xxxxxxxxxxxxxxxxx'

In [43]:
config["model"]["compute_confidence"] = True

In [44]:
agent = LabelingAgent(config=config)

In [45]:
agent.plan('test.csv')

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $7.6114 │
│ Number of Examples       │ 2000    │
│ Average cost per example │ $0.0038 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at answering questions based on wikipedia articles. Your job is to answer the following questions using the context provided with the question. Use the context to answer the question - the answer is a continuous span of words from the context.


Your answer will consist of an explanation, followed by the correct answer. The last line of the response should always be is JSON format with one key: {"label": "the correct answer"}.
 If the question cannot be answered using the context and the context alone without any outside knowledge, the question is unanswerable. If the question is unanswerable, return the answer as {"label": "unanswerable"}


Some examples with their output answers are provided below:

Context: The final major evolution of the steam engine design was the use of steam turbines starting in the late part of the 19th century. Steam turbines are generally more efficient than reciprocating piston type steam engines (for outputs above several hundred horsepow

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [46]:
labels, df, metrics_list = agent.run('test.csv', max_items=100)

2023-06-14 16:33:52 autolabel.labeler INFO: Task run already exists.


There is an existing task with following details: id='1525789836' created_at=datetime.datetime(2023, 6, 14, 16, 31,
48, 957264) task_id='10edc0f4575b5cb8fb15510f02fb5c0a' dataset_id='78bf0a6a6faae8cff14a5e36a12d0e2a' 
current_index=100 output_file='test_labeled.csv' status=<TaskStatus.ACTIVE: 'active'> error=None metrics=None

Evaluating the existing task...

Metric: auroc: 0.5


┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ f1     ┃ support ┃ threshold ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 0.5474 │ 100     │ -inf      │ 0.21     │ 1.0             │
│ 0.5474 │ 100     │ 0.5       │ 0.21     │ 1.0             │
└────────┴─────────┴───────────┴──────────┴─────────────────┘

...

2023-06-14 16:38:35 openai INFO: error_code=None error_message='The server had an error while processing your request. Sorry about that!' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
2023-06-14 16:38:35 langchain.embeddings.openai WARNING: Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
2023-06-14 16:38:40 openai INFO: error_code=None error_message='The server had an error while processing your request. Sorry about that!' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
2023-06-14 16:38:40 langchain.embeddings.openai WARNING: Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
2023-06-14 16:39:50 

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Metric: auroc: 0.9204
Actual Cost: 0.0113


┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ f1     ┃ support ┃ threshold ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 0.5474 │ 100     │ -inf      │ 0.21     │ 1.0             │
│ 1.0    │ 1       │ 0.9996    │ 1.0      │ 0.01            │
│ 1.0    │ 14      │ 0.9872    │ 1.0      │ 0.14            │
│ 0.8549 │ 18      │ 0.9652    │ 0.7778   │ 0.18            │
│ 0.8625 │ 19      │ 0.9532    │ 0.7895   │ 0.19            │
│ 0.8516 │ 20      │ 0.95      │ 0.75     │ 0.2             │
│ 0.8586 │ 21      │ 0.9486    │ 0.7619   │ 0.21            │
│ 0.8609 │ 22      │ 0.9291    │ 0.7273   │ 0.22            │
│ 0.867  │ 23      │ 0.9281    │ 0.7391   │ 0.23            │
│ 0.8026 │ 33      │ 0.8811    │ 0.5152   │ 0.33            │
│ 0.8084 │ 34      │ 0.876     │ 0.5294   │ 0.34            │
│ 0.7026 │ 46      │ 0.8503    │ 0.3913   │ 0.46            │
│ 0.709  │ 47      │ 0.8485    │ 0.4043   │ 0.47            │
│ 0.6957 │ 51      │ 0.8382    │ 0.3725   │ 0.51            │
│ 0.7015 │ 52      │ 0.8378    │ 0.3846   │ 0.52            │
│ 0.6701 │ 61      │ 0.8199    │ 0.3279   │ 0.61            │
│ 0.6755 │ 62      │ 0.8198    │ 0.3387   │ 0.62            │
│ 0.5474 │ 100     │ 0.0       │ 0.21     │ 1.0             │
└────────┴─────────┴───────────┴──────────┴─────────────────┘

Total number of failures: 0

Looking at the table above, we can see that if we set the confidence threshold at `0.9281`, we are able to label at 73.9% accuracy and getting a completion rate of 23%. This means, we would ignore all the data points where confidence score is less than `0.9281` (which would end up being around 77% of all samples). This would, however, guarantee a very high quality labeled dataset for us. 